In [6]:
#https://youtu.be/TIU2PwSoDS8
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import nltk
import re

In [7]:
data = pd.read_csv("./data/lenta-ru-news.csv", low_memory=False)

In [8]:
#data.head(5)

In [9]:
topics = ['Путешествия', 'Ценности', 'Мир', 'Наука и техника', 'Экономика']
news_in_cat_count = 1000

In [10]:
df_res = pd.DataFrame()
#print(def_res)
for topic in tqdm(topics):
    df_topic = data[data["topic"]==topic][:news_in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
df_res
#print(df_res)
#df_res.head(3)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2014/06/26/annoyingphotos/,Составлен рейтинг самых раздражающих отпускных...,"Названы самые раздражающие фотографии, которые...",Путешествия,Мнения,2014/06/26
1,https://lenta.ru/news/2015/01/09/flight/,В Европе появится 10-минутный авиарейс,Австрийская авиакомпания FlyNiki открывает 10-...,Путешествия,Мир,2015/01/09
2,https://lenta.ru/news/2015/01/28/tsipras/,Греческий премьер призвал избавиться от отелей...,Премьер-министр Греции Алексис Ципрас призвал ...,Путешествия,Мир,2015/01/28
3,https://lenta.ru/news/2015/02/03/standfly/,Китайская авиакомпания собралась ввести в само...,Китайский лоукостер Spring Airlines собрался п...,Путешествия,Мир,2015/02/03
4,https://lenta.ru/news/2015/02/12/tripforvalent...,Россияне отправятся отмечать День святого Вале...,Большинство российских путешественников отметя...,Путешествия,Россия,2015/02/12
...,...,...,...,...,...,...
4995,https://lenta.ru/news/2000/04/07/prices/,"В марте цены выросли на 0,6 процента",Потребительские цены на товары и услуги для на...,Экономика,Все,2000/04/07
4996,https://lenta.ru/news/2000/04/07/mrot/,Госдума увеличила минимальный размер оплаты труда,Госдума в пятницу утвердила закон о повышении ...,Экономика,Все,2000/04/07
4997,https://lenta.ru/news/2000/04/07/kamaz/,ЕБРР согласен реструктурировать долги КамАЗа,Европейский банк реконструкции и развития (ЕБР...,Экономика,Все,2000/04/07
4998,https://lenta.ru/news/2000/04/07/market/,Московскими рынками будет заведовать отдельная...,"Все дела, связанные с уплатой налогов московск...",Экономика,Все,2000/04/07


In [12]:
texts = df_res['text']
#print(texts[0])
#type(texts)

In [13]:
#print(texts[0])

In [14]:
from nltk.stem.snowball import SnowballStemmer
stemer = SnowballStemmer("russian")

In [15]:
def token_and_stem(text):
    sent_tokenize=nltk.sent_tokenize(text)
   
    #tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = [word for sent in sent_tokenize for word in nltk.word_tokenize(sent)]
    
    #print(tokens)
    filtered_tokens=[]
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemer.stem(t) for t in filtered_tokens]
    return stems

In [25]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words("russian")
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])


#print(stopwords)
#print("*****************************************************************************************************")
#token_and_stem(texts[5])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kl1356\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kl1356\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, 
                                   min_df=0.01, 
                                   max_features=100000, 
                                   stop_words=stopwords,
                                   use_idf=True, 
                                   tokenizer=token_and_stem, 
                                   ngram_range=(1,3)
                                  )

In [30]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

Wall time: 48 s


In [31]:
#print(tfidf_matrix)
texts.shape

(5000,)

In [32]:
num_clusters = 5
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)

In [33]:
%%time
km.fit(tfidf_matrix)


Wall time: 5.3 s


KMeans(n_clusters=5)

In [34]:
idx=km.fit(tfidf_matrix)
clusterkm=km.labels_.tolist()
frame=pd.DataFrame(texts)

out = { 'text': texts, 'cluster': clusterkm, 'topic': df_res['topic'] }
frame1 = pd.DataFrame(out, columns = ['text', 'cluster', 'topic'])

In [35]:
#frame1[96:99]
#print(frame1.head(10))
fp = frame1[ frame1.topic.eq('Путешествия') & frame1.cluster.eq(1) ]
print(len(fp))

0


In [36]:
# MiniBatchKMeans

#from sklearn.cluster import MiniBatchKMeans

#mbk  = MiniBatchKMeans(init='random', n_clusters=num_clusters) #(init='k-means++', ‘random’ or an ndarray)\n",
#mbk.fit_transform(tfidf_matrix)
#%time mbk.fit(tfidf_matrix)
#miniclusters = mbk.labels_.tolist()
#print (mbk.labels_)

In [37]:
#ser = data[ data['topic'] == 'Путешествия']['title']
#ser

In [38]:
#print(topics)
#print(num_clusters)

In [39]:
for topic in topics:
    print(topic, end = ' \t\t ')
    for cluster in range(num_clusters):
        print(cluster, ':', len(frame1[ frame1.topic.eq(topic) &  frame1.cluster.eq(cluster) ]), end = '   ')
    print()

Путешествия 		 0 : 237   1 : 0   2 : 12   3 : 750   4 : 1   
Ценности 		 0 : 163   1 : 803   2 : 21   3 : 12   4 : 1   
Мир 		 0 : 905   1 : 0   2 : 64   3 : 0   4 : 31   
Наука и техника 		 0 : 593   1 : 1   2 : 16   3 : 0   4 : 390   
Экономика 		 0 : 137   1 : 0   2 : 856   3 : 4   4 : 3   


In [40]:
clust_res = pd.DataFrame(columns=['topic', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4'])
i = 0
for topic in topics:
    cnt = []
    for cluster in range(num_clusters):
        cnt.append(len(frame1[ frame1.topic.eq(topic) &  frame1.cluster.eq(cluster) ]))
    clust_res.loc[i] = [topic] + cnt
    i += 1
clust_res

,topic,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
0,Путешествия,237,0,12,750,1
1,Ценности,163,803,21,12,1
2,Мир,905,0,64,0,31
3,Наука и техника,593,1,16,0,390
4,Экономика,137,0,856,4,3


In [41]:
tech_text = '''
    "Ученые Университета Аризоны в США открыли неизвестный принцип эволюции, согласно которому естественный отбор не может благоприятствовать сразу нескольким полезным мутациям, повышающим приспособленность организма к окружающей среде. Вместо этого он фокусируется только на том, что достаточно для выживания существа, и игнорирует другие возможности для улучшения. Об этом сообщается в статье в журнале Proceedings of the National Academy of Sciences.
    Исследователи спровоцировали несколько мутаций, которые «ломали» механизм трансляции у шести различных штаммов бактерии кишечной палочки Escherichia coli. Трансляция — это процесс, при котором заложенная в генах информация о нуклеотидной последовательности белков переносится к рибосомам и участвует в белковом синтезе. Известно, что более чем за 3,5 миллиарда лет эволюции механизм трансляции не претерпел существенных изменений.
    Ряд нефатальных мутаций, затрагивающих функционирование трансляционного процесса, должен с течением времени быть исправлен эволюцией. Для этого ученые заставили штаммы бактерий с различными изменениями конкурировать друг с другом, благодаря чему осуществлялся естественный отбор. Со временем внутри популяции накапливались полезные мутации, которые могли быть сохранены и использованы для улучшения трансляции.'''

In [42]:
text_vec = tfidf_vectorizer.transform([tech_text])
pd.DataFrame.sparse.from_spmatrix(text_vec)

,0,1,2,3,4,5,6,7,8,9,...,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
km.predict(text_vec)

array([0])

In [44]:
econ_text = '''
    Доллар США считается символом надежности, безопасности и экономического процветания.
    Он занимает неоспоримое доминирующее положение в международной финансовой системе с середины XX века и производит
    впечатление непобедимого титана. Однако эра господства доллара как основной мировой резервной валюты медленно
    подходит к концу. Крупнейшие банки предрекают ему резкий спад уже в следующем году, а известный экономист
    Стивен Роуч уверен, что американская валюта может обесцениться на треть.
    Причинами обвала станут сокращение сбережений населения, рост государственного долга США и усиление Китая.
    '''

In [45]:
text_vec = tfidf_vectorizer.transform([econ_text])
km.predict(text_vec)

array([2])